## Importing necessary libraries

In [1]:
import numpy as np
import cv2

# For ML model 
import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model

## Model Initialization

In [2]:
# Initializing Mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_Draw = mp.solutions.drawing_utils

## Import models and classes

In [3]:
# Loading the  gesture Control Model
model = load_model("mp_hand_gesture")

#Load class names
f = open('gesture.names' , 'r')
classNames = f.read().split('\n')
f.close()
print(classNames)

['okay', 'peace', 'thumbs up', 'thumbs down', 'call me', 'stop', 'rock', 'live long', 'fist', 'smile']


### Reading frames from webcam

In [4]:
# Initialize webcam for hand gestures
cap = cv2.VideoCapture(0)
while True:
    # Read each frame
    _, frame = cap.read()
    
    # Get frame dimensions
    x, y, c = frame.shape
    
    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get hand landmarks prediction
    result = hands.process(frame_rgb)
    
    className = ''
    
    # Post process the result
    if result.multi_hand_landmarks:
        landmarks = []
        for hand_landmarks in result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                lm_x = int(lm.x * x)
                lm_y = int(lm.y * y)
                landmarks.append([lm_x, lm_y])
            
            # Draw landmarks on the frame
            mpDraw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)
            
            # Predict gesture 
            prediction = model.predict([landmarks])  
            classID = np.argmax(prediction)
            className = classNames[classID]
            
            # Show the prediction on the frame
            cv2.putText(frame, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (0, 0, 255), 2, cv2.LINE_AA)
    
    # Show the final output
    cv2.imshow("Output", frame)
    
    # Press 'q' to exit the window
    if cv2.waitKey(1) == ord('q'):
        break

# Release the webcam and destroy all active windows
cap.release()
cv2.destroyAllWindows()

NameError: name 'mpDraw' is not defined